In [1]:
# important for gpuhub
# !pip install -r ../../requirements.txt --upgrade

## Importing Libraries and tokens

In [2]:
import wandb
import os
import pickle
import json
import torch

# load .env file
from dotenv import load_dotenv
from wandb_downloader import WandbDownloader
from geo_model_tester import GeoModelTester
from image_data_handler_test import TestImageDataHandler
from best_run_loader import BestRunLoader

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
WANDB_TOKEN = os.getenv('WANDB_TOKEN')
# Define where to run
env_path = '../../.env'
if not WANDB_TOKEN and os.path.exists(env_path):
  load_dotenv(env_path)
  WANDB_TOKEN = os.getenv('WANDB_TOKEN')

In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    
    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


## Loading files from wandb

In [5]:
wandb.login(key=WANDB_TOKEN) if WANDB_TOKEN else wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luki-st (nlp_ls). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/lukasstoeckli/.netrc


True

In [6]:
# Example usage:
entity = "nlp_ls"
predict_coordinates=False
predict_regions=True
project = "dspro2-predicting-region" if predict_regions else ("dspro2-predicting-coordinates" if predict_coordinates else "dspro2-predicting-country")
metric_name = "Validation Accuracy Top 1" if not predict_coordinates else "Validation Distance (km)"
metric_ascending = False if not predict_coordinates else True
data_augmentation = "base_augmentation" # or "full_augmentation_v2"
datasize = 332786  # Replace with the desired datasize
file_names_to_download = [".pth", ".json"]
image_size = [80, 130]

downloader = WandbDownloader(entity, project, data_augmentation, datasize, image_size)
run_data = downloader.get_and_collect_best_runs(metric_name, file_names_to_download, metric_ascending=metric_ascending)

# Now run_data contains all necessary information for further processing
import pprint
pprint.pprint(run_data)

dspro2-predicting-region: Found 5 matching runs for datasize 332786 and base_augmentation.
{'Best Run 1': {'files': {'best_model': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/best_model_checkpointmodel_efficientnet_b1_lr_0.01_opt_adamW_weightDecay_0.01_imgSize_[80, '
                                        '130]_predict_coordinates_False.pth',
                          'country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/country_to_index.json',
                          'region_index_to_country_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_index_to_country_index.json',
                          'region_index_to_middle_point.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_index_to_middle_point.json',
                          'region_to_index.json': 'https://a

In [7]:
run_data.keys()

dict_keys(['Best Run 1', 'Best Run 2', 'Best Run 3', 'Best Run 4', 'Best Run 5'])

In [8]:
# Print the validation accuracy for the top 1, 3, and 5 predictions
if predict_coordinates:
    print(f"Validation Distance (km): ", run_data["Best Run 1"]["metrics"]["Validation Distance (km)"])
else:
  for i in [1, 3, 5]:
      print(f"Validation Accuracy Top {i}: ", run_data["Best Run 1"]["metrics"][f"Validation Accuracy Top {i}"])

Validation Accuracy Top 1:  0.2021875985996965
Validation Accuracy Top 3:  0.34047508150908246
Validation Accuracy Top 5:  0.4164250191565125


In [9]:
run_data["Best Run 1"]["parameters"]

{'seed': 42,
 'epochs': 50,
 'optimizer': 'adamW',
 'model_name': 'efficientnet_b1',
 'mapped_data': False,
 'dataset_size': 332786,
 'weight_decay': 0.01,
 'learning_rate': 0.01,
 'predict_regions': True,
 'input_image_size': [80, 130],
 'data_augmentation': 'base_augmentation',
 'different_regions': 4596,
 'dataset_identifier': '22a493044dbe99c1d431b9ee4656792efbb09ece4182274670ba5faec505d9cf',
 'different_countries': 138,
 'predict_coordinates': False}

In [10]:
run_data["Best Run 1"]["files"]

{'wandb_manifest.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/artifact/918441477/wandb_manifest.json',
 'best_model': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/best_model_checkpointmodel_efficientnet_b1_lr_0.01_opt_adamW_weightDecay_0.01_imgSize_[80, 130]_predict_coordinates_False.pth',
 'country_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/country_to_index.json',
 'region_index_to_country_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_index_to_country_index.json',
 'region_index_to_middle_point.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_index_to_middle_point.json',
 'region_to_index.json': 'https://api.wandb.ai/files/nlp_ls/dspro2-predicting-region/kwlp08u6/run-20240623_193550-kwlp08u6/region_to_index.json',
 'wandb-metada

## Loading metrics for best runs for each datasize, with and without augmentation

In [11]:
# Create configuration entries
def create_configurations(names, sizes, augmentations, image_size):
    configurations = []
    for name in names:
        for size in sizes:
            for aug in augmentations:
                configurations.append({
                    "project": f"dspro2-predicting-{name}",
                    "data_augmentation": aug,
                    "datasize": size,
                    "image_size": image_size
                })
    return configurations

# Load best runs
def load_best_runs(entity, project, metric_name, data_augmentation, datasize, image_size, file_names_to_download):
    metric_ascending = False
    if project.endswith("coordinates"):
      metric_ascending = True
    downloader = WandbDownloader(entity, project, data_augmentation, datasize, image_size)
    return downloader.get_and_collect_best_runs(metric_name, file_names_to_download, metric_ascending=metric_ascending)

# Load best runs wit the configurations
def load_results(entity, configurations, metric_name, file_names):
    results = {}
    for configs in configurations.values():
        for config in configs:
            key = f"{config['project']}_{config['data_augmentation']}_{config['datasize']}_{config['image_size']}"
            results[key] = load_best_runs(
                entity,
                config['project'],
                metric_name,
                config['data_augmentation'],
                config['datasize'],
                config['image_size'],
                file_names
            )
    return results

In [12]:
# Example usage:
entity = "nlp_ls"
metric_name = "Validation Accuracy Top 1"
project_names = ["coordinates", "region", "country"]
file_names_to_download = [".pth", ".json"]

loader = BestRunLoader(entity, metric_name, project_names, file_names_to_download)

# See the keys of the results
print("\n", loader.results.keys(),"\n")

# See the keys of the results
print(loader.results["dspro2-predicting-region_base_augmentation_81505_(80, 130)"].keys())

dspro2-predicting-coordinates: No matching runs found for datasize 81505 and base_augmentation.
dspro2-predicting-coordinates: No matching runs found for datasize 81505 and full_augmentation_v2.
dspro2-predicting-coordinates: Found 5 matching runs for datasize 332786 and base_augmentation.
dspro2-predicting-coordinates: No matching runs found for datasize 332786 and full_augmentation_v2.
dspro2-predicting-coordinates: No matching runs found for datasize 79000 and base_augmentation.
dspro2-predicting-region: Found 5 matching runs for datasize 81505 and base_augmentation.
dspro2-predicting-region: Found 2 matching runs for datasize 81505 and full_augmentation_v2.
dspro2-predicting-region: Found 5 matching runs for datasize 332786 and base_augmentation.
dspro2-predicting-region: Found 2 matching runs for datasize 332786 and full_augmentation_v2.
dspro2-predicting-region: No matching runs found for datasize 79000 and base_augmentation.
dspro2-predicting-country: No matching runs found for 

In [13]:
# Count runs per project
project_run_counts = loader.count_runs_per_project()
print(f"Found runs for each project", project_run_counts)

Found runs for each project {'dspro2-predicting-coordinates': 5, 'dspro2-predicting-region': 14, 'dspro2-predicting-country': 3}


## Loading data and creating data loader

In [ ]:
cache = True

run = None

for i in range(5):
    run = run_data[f"Best Run {i+1}"]
    if run["files"].get("test_data", None) and run["files"].get("best_model", None):
        break
    else:
        run = None
        print(f"Run {i+1} does not contain the necessary files. Trying the next run...")
        
if run is None:
  raise Exception("No run with the necessary files found.")

# Creating Dataloaders with the classes
test_dataset = run["files"]['test_data']
files = run["files"]
country_to_index = files.get('country_to_index.json', None)
region_to_index = files.get('region_to_index.json', None)
region_index_to_middle_point = files.get('region_index_to_middle_point.json', None)
region_index_to_country_index = files.get('region_index_to_country_index.json', None)

data_handler = TestImageDataHandler(test_dataset, country_to_index, region_to_index, region_index_to_middle_point, region_index_to_country_index, cache=cache)
test_dataloader = data_handler.test_loader
country_to_index = data_handler.country_to_index
region_to_index = data_handler.region_to_index
region_index_to_middle_point = data_handler.region_index_to_middle_point
region_index_to_country_index = data_handler.region_index_to_country_index

num_regions = data_handler.num_regions
num_countries = data_handler.num_countries

## Evaluating the model

In [ ]:
num_classes = 3 if predict_coordinates else (num_regions if predict_regions else num_countries)

if num_classes == 0:
    raise ValueError("No classes detected. Please check the data.")

geo_model_tester = GeoModelTester(
    test_dataloader=test_dataloader, 
    num_classes=num_classes, 
    predict_coordinates=predict_coordinates, 
    country_to_index=country_to_index,
    region_to_index=region_to_index,
    region_index_to_middle_point=region_index_to_middle_point,
    region_index_to_country_index=region_index_to_country_index,
    predict_regions=predict_regions if not predict_coordinates else None
)

In [ ]:
# TODO: Test the model from best runs
# TODO: Show the different models with the best results (also do it for different data sizes and mapped/non-mapped data)
model_name = run["parameters"]["model_name"]
pretrained_weights = run["files"]['best_model']

# Countries from 81k more mapped dataset
countries_only = ["Albania", "Argentina", "Australia", "Austria", "Bangladesh", "Belgium", "Bolivia, Plurinational State of", "Botswana", "Brazil", "Bulgaria", "Cambodia", "Canada", "Chile", "Colombia", "Croatia", "Czechia", "Denmark", "Dominican Republic", "Ecuador", "Estonia", "Eswatini", "Finland", "France", "Germany", "Ghana", "Greece", "Guatemala", "Hungary", "India", "Indonesia", "Ireland", "Israel", "Italy", "Japan", "Kenya", "Korea, Republic of", "Kyrgyzstan", "Lao People's Democratic Republic", "Latvia", "Lesotho", "Lithuania", "Malaysia", "Malta", "Mexico", "Montenegro", "Netherlands", "New Zealand", "Nigeria", "North Macedonia", "Norway", "Peru", "Philippines", "Poland", "Portugal", "Romania", "Russian Federation", "Rwanda", "Senegal", "Serbia", "Singapore", "Slovakia", "Slovenia", "South Africa", "Spain", "Sri Lanka", "Sweden", "Switzerland", "Thailand", "T\u00fcrkiye", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay"]

geo_model_tester.test(model_type=model_name, model_path=pretrained_weights, balanced_on_countries_only=countries_only, accuracy_per_country=False)

if not predict_coordinates:
  # And over all countries
  geo_model_tester.test(model_type=model_name, model_path=pretrained_weights, balanced_on_countries_only=None, accuracy_per_country=True)